[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import pandas as pd
import requests
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

In [ ]:
%%ampl_eval
set STOCKS;
set ACCOUNTS = {1..2};
set SECTORS;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set STOCKS_IN_SECTOR {SECTORS} within STOCKS;
set PENALTIES = {"concentration", "sector"};

param base {ACCOUNTS};
param market_value {STOCKS};
param sign {s in STOCKS} = if market_value[s] >= 0 then 1 else -1;
param volume {STOCKS};
param sizes {PENALTIES, ACCOUNTS} default 0 >= 0;
set ACCOUNTS_WITH_CONC = {a in ACCOUNTS: sizes["concentration", a]>0};
set ACCOUNTS_WITH_SECT = {a in ACCOUNTS: sizes["sector", a]>0};
param breaks {p in PENALTIES, a in ACCOUNTS, i in 1..sizes[p, a]}
          >= if i = 1 then 0.0 else breaks[p, a, i-1];
param slopes {p in PENALTIES, a in ACCOUNTS, i in 1..sizes[p, a]}
          > if i = 1 then 0.0 else slopes[p, a, i-1];

minimize TotalCost;

subject to Completeness {s in STOCKS}:
          to_come = market_value[s];

var X {s in STOCKS, ACCOUNTS}
          >= <<0; 1,0>> market_value[s], <= <<0; 0,1>> market_value[s],
          coeff Completeness[s] 1;

subject to AbsXLevel {s in STOCKS, a in ACCOUNTS}:
          to_come = sign[s] * X[s, a];

var AbsX {s in STOCKS, a in ACCOUNTS} >= 0,
          obj TotalCost base[a],
          coeff AbsXLevel[s, a] 1;

subject to GMVLevel {a in ACCOUNTS}:
          to_come = sum {s in STOCKS} AbsX[s, a];

var GMV {a in ACCOUNTS} >= 0,
          coeff GMVLevel[a] 1;

subject to ConcentrationLevel {s in STOCKS, a in ACCOUNTS_WITH_CONC}:
          to_come = AbsX[s, a] - 0.05 * GMV[a];

subject to SectorMVLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come = sum {s in STOCKS_IN_SECTOR[sector]} X[s, a];

var SectorMV {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          coeff SectorMVLevel[sector, a] 1;

subject to SectorNMVDomain {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come >= <<0; -1,1>> SectorMV[sector, a];

var SectorNMV {sector in SECTORS, a in ACCOUNTS_WITH_SECT} >= 0,
          coeff SectorNMVDomain[sector, a] 1;

subject to SectorExcessLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come = SectorNMV[sector, a] - 0.2 * GMV[a];

var Concentration {s in STOCKS, a in ACCOUNTS_WITH_CONC},
          coeff ConcentrationLevel[s, a] 1;

subject to ConcentrationPenaltyLevel {s in STOCKS, a in ACCOUNTS_WITH_CONC}:
          to_come >= <<{p in 1..sizes["concentration", a]} breaks["concentration", a, p];
                       {p in 0..sizes["concentration", a]} if p = 0 then 0.0 else slopes["concentration", a, p]>> Concentration[s, a];

var ConcentrationPenalty {s in STOCKS, a in ACCOUNTS_WITH_CONC},
          obj TotalCost 1,
          coeff ConcentrationPenaltyLevel[s, a] 1;

var SectorExcess {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          coeff SectorExcessLevel[sector, a] 1;

subject to SectorPenaltyLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come >= <<{p in 1..sizes["sector", a]} breaks["sector", a, p];
                       {p in 0..sizes["sector", a]} if p = 0 then 0.0 else slopes["sector", a, p]>> SectorExcess[sector, a];

var SectorPenalty {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          obj TotalCost 1,
          coeff SectorPenaltyLevel[sector, a] 1;

In [ ]:
response = requests.get(
    "https://www.harborcapital.com/page-data/etf/lseq/page-data.json"
)
response.raise_for_status()
tabs = response.json()["result"]["data"]["contentstackProductV2"]["product_tabs"]
full_holdings = next(
    section for tab in tabs
    if (data_section := tab["data_section"])
    and data_section["title"] == "Holdings"
    for section in data_section["section"]
    if section["title"] == "Full Holdings"
)
holdings = (
    pd.json_normalize(full_holdings, ["api_reference", "data", "fullHoldings"])
    .set_index("ticker")[["marketValue"]]
    .rename(columns={"marketValue": "market_value"})
)

response = requests.get(
    "https://api.nasdaq.com/api/screener/stocks",
    params={
        "tableonly": False,
        "limit": 10_000,
        "download": True,
    },
    headers={"User-Agent": ""}
)
response.raise_for_status()
data = response.json()

screener = (
    pd.json_normalize(data, ["data", "rows"])
    .rename(columns=data["data"]["headers"])
    .set_index("Symbol")
    .astype({"Volume": float})[["Volume", "Sector"]]
    .rename(columns={"Volume": "volume"})
)
π = holdings.join(screener, how="inner")
π

In [ ]:
penalties = pd.DataFrame(
    [["conc", 1, 1, 0.0, 0.1], ["sect", 1, 1, 0.0, 0.1]],
    columns=["name", "account", "index", "breaks", "slopes"],
).set_index(["name", "account", "index"])
penalties

In [ ]:
ampl.set_data(π[["market_value", "volume"]], set_name="STOCKS")
ampl.param["base"] = {1: 0.05, 2: 0.06}
ampl.param["sizes"] = penalties["breaks"].groupby(["name", "account"]).count()
ampl.set_data(penalties)

stocks_in_sector = π.groupby("Sector").groups
ampl.set["SECTORS"] = stocks_in_sector.keys()
ampl.set["STOCKS_IN_SECTOR"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
X = ampl.get_variable("X").get_values().to_pandas()
π.assign(
    account_1=X.loc[(π.index, 1), "X.val"].to_numpy(),
    account_2=X.loc[(π.index, 2), "X.val"].to_numpy(),
)